In [1]:
import os
import hashlib
import shutil
import datetime
import time
import pickle

In [2]:
class Folder:
    def __init__(self, path):
        self.path = path
        self.files = {}
        self.folders = {}
        self.scan_folder()

    def scan_folder(self):
        obj = os.scandir(self.path)
        for entry in obj:
            try:
                if entry.is_dir() and "{p}" not in entry.name:
                    self.folders[entry.name] = Folder(os.path.join(self.path, entry.name))
                elif not entry.is_dir() and "{p}" not in entry.name:
                    self.files[entry.name] = self.md5_hashing(os.path.join(self.path, entry.name))
            except Exception as e:
                print(f"  Momentary Error {type(e)} has occurred in path: ", self.path)

    def md5_hashing(self, filename):
        md5_h = hashlib.md5()
        
        # open file for reading in binary mode
        with open(filename,'rb') as file:
        # read file in chunks and update hash
            chunk = 0
            while chunk != b'':
                chunk = file.read(1024) 
                md5_h.update(chunk)
        
        # return the hex digest
        return md5_h.hexdigest()


In [3]:
class Queue:

    
    def __init__(self, node, node_type=None):
        self.lis = [{"main":node}, {}]
        self.node_type = node_type
        
    
    def search(self, word, Similar=False):
        lis = []
        queue = [self.lis[0]["main"]]
        for i in queue:
            for k in i.files:
                if Similar:
                    if word in k:
                        lis.append(os.path.join(i.path,k))
                else:
                    if word in k.split("."):
                        lis.append(os.path.join(i.path,k))
                        
            for j in i.folders:
                if word == j:
                    lis.append(os.path.join(i.path,j))
                queue.append(i.folders[j])
        print("Search Complete!")
        return lis
                
    def add_nested_folders(self, word):
        try:
            node = self.lis[0][word]
            for i in node.folders:
                self.lis[1][i] = node.folders[i]
        except:
            print(f"Error occored in {self.node_type}.add_nested_folders(); The key {word} doesnt exist!")
            
    def shift(self,word):
        try:
            node = self.lis[0][word]
            self.lis[1][word] = node
        except:
            print(f"Error occored in {self.node_type}.shift(); The key {word} doesnt exist!")
    
    def flush(self):
        self.lis.pop(0)
        self.lis.append({})
         

In [7]:
class Backup:
    
    def __init__(self, from_path, to_path):
        self.from_folder = Folder(from_path)
        try:
            filename = os.path.join(to_path,"__backup_log__{p}","to_folder_tree.pkl")
            with open(filename, "rb") as file:
                self.to_folder = pickle.load(file)
        except:
            self.to_folder = Folder(to_path)

    def commence_backup(self):
        queue_from = Queue(self.from_folder,"from")
        queue_to = Queue(self.to_folder,"to")
        if self.from_folder.path != self.to_folder.path:
            while self.isEmpty(queue_from,queue_to):

                for folder in queue_from.lis[0]:
                    queue_from.add_nested_folders(folder)

                for folder in queue_to.lis[0]:
                    queue_to.add_nested_folders(folder)

                for folder in queue_from.lis[0]:
                    if folder in queue_to.lis[0]:
                        for j in queue_from.lis[0][folder].folders:
                            if j not in queue_to.lis[0][folder].folders:
                                try:
                                    shutil.copytree(os.path.join(queue_from.lis[0][folder].path,j), os.path.join(queue_to.lis[0][folder].path,j))
                                except:
                                    pass

                # compare first dictonary
                for folder in queue_from.lis[0]:
                    if folder in queue_to.lis[0]:
                        self.compare_files(queue_from.lis[0][folder],queue_to.lis[0][folder])

                if len(queue_to.lis[0])!=0:
                    for folder in queue_to.lis[0]:
                        queue_to.shift(folder)

                queue_from.flush()
                queue_to.flush()
        self.update_backup()
                
            
    def compare_files(self,node_from,node_to):
        files_from = node_from.files
        files_to = node_to.files
        for i in files_from:
            if i in files_to and files_from[i] == files_to[i]:
                continue
            else:
                shutil.copy(os.path.join(node_from.path,i),os.path.join(node_to.path,i))
                
    def isEmpty(self,queue_from,queue_to):
        if len(queue_from.lis[0]) == 0 and len(queue_from.lis[1]) == 0:
            return False
        else:
            return True
        
    def update_backup(self):
        try:
            os.mkdir(os.path.join(self.to_folder.path,"__backup_log__{p}"))
        except:
            pass
        filename = os.path.join(self.to_folder.path,"__backup_log__{p}","to_folder_tree.pkl")
        with open(filename, "wb") as file:
            pickle.dump(self.to_folder, file)
        
        file_path = os.path.join(self.to_folder.path,"__backup_log__{p}","__backup_log__.txt")
        file = open(file_path, "w")
        curr = datetime.date.today()
        curr = curr.strftime("%B %d, %Y")
        file.write(f"Last Backup Date: {curr}")
        file.close()


In [9]:
print("1. Perform Backup")
print("2. Perform Search")
while(True):
    n = input("Enter Choice: ")
    try:
        n = int(n)
    except:
        n = -1
    
    if n == 1:
        from_path = input("Enter From Path: ")
        to_path = input("Enter To Path:   ")
        start_time = time.time()
        print("Building Folder Tree...")
        # Initilaize Backup Object
        c = Backup(from_path,to_path)
        print("Commencing Backup...")
        c.commence_backup()
        end_time = time.time()
        print(f"Backup Successfully Completed in {int(round(end_time-start_time,3)//60)}min {round((end_time-start_time)%60,3)}sec!")
        break
    
    elif n == 2:
        from_path = input("Enter Path to Search: ")
        word = input("Enter Name of File or Folder: ")
        Similar = input("Need Similar Search Results? y/n: ")
        if Similar == 'y':
            Similar = True
        else:
            Similar = False
        
        start_time = time.time()
        try:
            filename = os.path.join(to_path,"__backup_log__{p}","to_folder_tree.pkl")
            with open(filename, "rb") as file:
                c = pickle.load(file)
            print("Loading Folder Tree...")
        except:
            print("Building Folder Tree...")
            c = Folder(from_path)
        
        print("Commencing Search...")
        c = Queue(c,"search")
        lis = c.search(word,Similar)
        if len(lis) != 0:
            for i in lis:
                if os.path.isdir(i):
                    print("Folder: ",i)
                else:
                    print("File:   ",i)
            break
        else:
            print("No Results were Found")
            break
        
    else:
        print("Enter a valid Input!")
        


1. Perform Backup
2. Perform Search
Enter Choice: 1
Enter From Path: E:\D Folder Backup
Enter To Path:   E:\D Folder Backup
Building Folder Tree...



KeyboardInterrupt



In [ ]:
D:\Documents and Project\Projects\Data Backup Project\TEST\from
D:\Documents and Project\Projects\Data Backup Project\TEST\to